In [ ]:
# https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial

In [ ]:
import pandas as pd

df = pd.read_csv("../data/simpsons_dataset.csv")

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna().reset_index(drop=True)
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
corpus = list(df["spoken_words"].values)

In [ ]:
import nltk
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
w_n_lemmatizer = WordNetLemmatizer() 

corpus = [[w_n_lemmatizer.lemmatize(word) for word in word_tokenize(document.lower()) 
            if not word in stop_words and word.isalnum()] 
          for document in corpus]

corpus[0]

In [ ]:
pip install gensim

In [ ]:
from gensim.models.phrases import Phrases, Phraser
phrases = Phrases(corpus, min_count=30, progress_per=10000)
print(phrases)

In [ ]:
import multiprocessing

from gensim.models import Word2Vec

In [ ]:
cores = multiprocessing.cpu_count()
cores

In [ ]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [ ]:
w2v_model.build_vocab(corpus, progress_per=10000)
w2v_model.corpus_count

In [ ]:
w2v_model.train(corpus, 
                total_examples=w2v_model.corpus_count, 
                epochs=30, 
                report_delay=1)

In [ ]:
# which will make the model much more memory-efficient:
w2v_model.init_sims(replace=True)

In [ ]:
w2v_model.wv.most_similar(positive=["homer"])

In [ ]:
w2v_model.wv.most_similar(positive=["marge"])

In [ ]:
w2v_model.wv.most_similar(positive=["bart"])

In [ ]:
w2v_model.wv.similarity("eliza", 'bart')

In [ ]:
w2v_model.wv.doesnt_match(['jimbo', 'milhouse', 'kearney'])

In [ ]:
# Which word is to woman as homer is to marge?
w2v_model.wv.most_similar(positive=["woman", "homer"], negative=["marge"], topn=3)

In [ ]:
w2v_model.wv.most_similar(positive=["woman", "bart"], negative=["man"], topn=3)

In [ ]:
w2v_model.wv.save_word2vec_format("../data/word2vec.vec")

In [ ]:
with open("../data/word2vec.vec", "r") as f:
    tmp = f.read().split("\n")

with open("../data/word2vec.tsv", "w") as t:
    for i in range(1,len(tmp)):
        splited = tmp[i].split(" ")
        t.write("\t".join(splited[1:]) + "\n")